<a href="https://colab.research.google.com/github/sscs-ose/sscs-ose-chipathon.github.io/blob/main/template_notebook_to_follow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Title: 11-bit differential ADC SAR


```
SSCS TC-OSE Team, February 2023
SPDX-License-Identifier: Apache-2.0
```


|Name|Email|Affiliation|IEEE Member|SSCS Member|
|:--:|:--:|:----------:|:----------:|:----------:|
|Fredy Enrique Segura Quijano|fsegura@uniandes.edu.co|Universidad de Los Andes|Yes|No|
|David Alejandro Reyes Gonzales|darge3t.uis@gmail.com|HCL-Brazil|No|No|
|Juan Andrés Lopez Cubides|juan.andres.lopez491@gmail.com|Universidad de Los Andes|Yes|No|
|Juan Sebastián Moya Baquero (Lead) <br />|jsmoya07@gmail.com|Universidad Industrial de Santander/Universidad de Los Andes|Yes|Yes|

**_Abstract_** - This electronic document is a “live” template and already defines the components of your paper [title, text, heads,
etc.] in its style sheet. *

**_Key words_** - component; formatting; style; styling; insert (key words)


## Tool Installation

This is where you need to install your tools. We provide here an example where conda environment is being installed and then Ngspice for simulations.


**_Tool setup adopted from @proppy and @bmurmann (see this [Colab notebook](https://colab.research.google.com/gist/proppy/a0c5ed3e28e942f1621200dcf67bad5a/sky130-pyspice-playground.ipynb#scrollTo=q0XHBAt1jGmQ))_**

In [ ]:
#@title Install dependencies {display-mode: "form"}
#@markdown - Click the ▷ button to setup the digital design environment based on [conda-eda](https://github.com/hdl/conda-eda).

ngspice_version = 'latest' #@param {type:"string"}
gf180mcu_fd_pr_version = 'latest' #@param {type:"string"}

if ngspice_version == 'latest':
  ngspice_version = ''

if gf180mcu_fd_pr_version == 'latest':
  gf180mcu_fd_pr_version = 'main'

import os
import pathlib
import urllib.request

!curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xj bin/micromamba
conda_prefix_path = pathlib.Path('conda-env')
CONDA_PREFIX = str(conda_prefix_path.resolve())
!bin/micromamba create --yes --prefix $CONDA_PREFIX
!echo 'python ==3.7*' >> {CONDA_PREFIX}/conda-meta/pinned
!CI=0 bin/micromamba install --yes --prefix $CONDA_PREFIX \
                     --channel litex-hub \
                     --channel main \
                     ngspice={ngspice_version}

ngspice_models_dir = pathlib.Path('globalfoundries-pdk-libs-gf180mcu_fd_pr/models/ngspice')
ngspice_models_dir.mkdir(exist_ok=True, parents=True)
models = ['design.ngspice', 'sm141064.ngspice', 'sm141064_mim.ngspice', 'smbb000149.ngspice']
for m in models:
  url = f'https://github.com/efabless/globalfoundries-pdk-libs-gf180mcu_fd_pr/raw/{gf180mcu_fd_pr_version}/models/ngspice/{m}'
  print('downloading:', url)
  with urllib.request.urlopen(url) as src:
    with (ngspice_models_dir / m).open('wb') as dst:
      dst.write(src.read())

PATH = os.environ['PATH']
%env CONDA_PREFIX={CONDA_PREFIX}
%env PATH={CONDA_PREFIX}/bin:{PATH}

## I. Introduction

Follows the proposal of an 11-bit SAR ADC for the SSCS "PICO" Open-Source Chipathon developed by different Colombian engineers from academia and industry.

Follows a description of the main specifications defined for the SAR-ADC:

|Specification|Symbol|Baseline requirement|Comment|
|:--:|:--:|:----------:|:----------:|
|Sampling rate|fs|≥ 1.5MS/s|||
|Effective number of bits|ENOB|≥ 9|Measured near Nyquist|
|Input Capacitance|Cin|≤ 5pF||

We know that ENOB is defined by:
ENOB = (SNDR - 1.76dB)/6.02

Then, SNDR --> 6.02 x ENOB + 1.76dB

The next table presents the relation between ENOB and SNDR based on the equation above.

|ENOB|SNDR|
|:--:|:--:|
|8|49.92|
|9|55.94|
|10|61.96|
|11|67.98|
|12|74|

Based on the information presented above and the specification of the contest, we must have an SNDR ≥ 55.94

We base our project on the following Master's Degree Dissertation:

- Reyes Gonzalez, D. A. Projeto de um conversor analógico-digital para um receptor UWB aplicado na detecção de câncer de mama em tecnologia CMOS (Master Thesis, Universidade de São Paulo).

We develop the design of the ADC based on the following considerations regarding the tracking process of the SAR:

1) Finite resistance of the switch.
2) Nyquist frequency.
3) Switch stabilization time.
4) Thermal Noise.
5) Charge injection --> Charge is stored in the channel when the switch is in strong inversion.
6) Clock Feedthrough.
7) Feedthrough.

We develop the design of the ADC based on the following considerations regarding the quantization process of the SAR:

1) Dynamic comparator since power consumption only occurs during the rising and falling edges.
2) Kickback noise in the dynamic comparator.
3) Static offset in the dynamic comparator.
4) Thermal noise.
5) Trade-off between thermal noise and power.
6) Metastability for small differential input voltages.
7) Conversion speed of the asynchronous operation is higher than the synchronous one.

The proposed ADC-SAR design is presented in the following figure.

![ADC_SAR proposal](./Images/SAR_ADC.png)


The ADC-SAR proposed is composed of the following sub-circuits:

1) Differential Capacitive DAC (blue rectangle).
2) Switches for sampling (red rectangle).
3) Bridge Capacitor (horizontal capacitor in the differential capacitive DAC).
4) Switches for tracking (green rectangle).
5) Dynamic Comparator (purple rectangle).
6) SAR Logic (grey square).
7) Asynchronous operation to avoid the use of a high-frequency clock.



## II. Implementation Details of your Idea

Once with defined the main architecture, we present in more detail the schematic selected for the sub-blocks that integrate de SAR-ADC.

**Sampling Switches**
We use the capacitor-merged circuit for the sampling switches, as presented in the following reference:

- Hariprasath, V., Guerber, J., Lee, S. H., & Moon, U. K. (2010). Merged capacitor switching based SAR ADC with highest switching energy-efficiency. Electronics letters, 46(9), 620.

--> The 11-bit ADC uses a 10-bit DAC. Follows a 3-bit schematic using the merged capacitors.

![3-bit schematic example for the sampling phase](./Images/Sampling_phase.png)

We define the following constant voltages:
1) Vcm = Vdd/2
2) Vrefp = Vdd
3) Vrefn = 0 V

Since the common mode voltage of the outputs of the differential capacitive DAC (vdp and vdn) is constant, the comparator offset is also constant and does not depend on the input voltage [2010_Liu].

- [2010_Liu]Liu, C. C., Chang, S. J., Huang, G. Y., & Lin, Y. Z. (2010). A 10-bit 50-MS/s SAR ADC with a monotonic capacitor switching procedure. IEEE Journal of Solid-State Circuits, 45(4), 731-740.

**Bootstrap circuit**

To solve the problem regarding injection charge and finite resistance, we use the bootstrap technique proposed in the references and illustrated in the Figure below:

- Hariprasath, V., Guerber, J., Lee, S. H., & Moon, U. K. (2010). Merged capacitor switching based SAR ADC with highest switching energy-efficiency. Electronics letters, 46(9), 620.

- Liu, C. C., Chang, S. J., Huang, G. Y., & Lin, Y. Z. (2010). A 10-bit 50-MS/s SAR ADC with a monotonic capacitor switching procedure. IEEE Journal of Solid-State Circuits, 45(4), 731-740.

![Sample Circuit with bootstrap technique.](./Images/Bootstrap.png)

Moreover, to protect the transistor terminal voltages against high-voltage values, as is the case with gate oxide breakdown or reverse breakdown voltages, the bootstrap technique is used to guarantee that the maximum terminal-to-terminal device voltages are Vdd. With this bootstrap technique, we increase device reliability during operation.

When the hold mode is enabled, the signal clks = '0'. Thus transistors M1, M3, M4, M7, and Ms are disabled. The capacitor Cb is charged to the Vdd value. 

When the track mode is enabled, the signal clks = '1'. Thus transistors M2, M5, M8, and M9 are disabled. Transistor Ms is on, which allows sampling of the signal at the input Vinp. The equivalent circuit of the track mode is presented in b) in the figure below.

![Equivalent Circuits for Track and Hold using the bootstrap technique.](./Images/Track_Hold_equivalents.png)


We consider the minimum SNDR for track and hold, to define the dimensions of the sampling switches.

SNDR_T_&_H = 6.02 x (N+2) + 1.76dB.

For the 11-bit ADC, we have an SNDR_T_&_H = 80.02 dB, therefore the ENOB is desired to be at least 13 bits. 


**Diferential track & hold circuit**
Follows the figure associated with the sub-circuit proposal. As mentioned before, the ADC is differential. We use dummy transistors with the same dimensions as the sampling transistors to mitigate the feedthrough effect when the circuit is during the hold phase.

![Differential track and hold circuit.](./Images/Differential_T_H_circuit.png)

**Diferential track & hold circuit**
Follows the figure associated with the sub-circuit proposal. In this figure, it is possible to observe the single-ended circuit associated with the equivalent MSB and LSB capacitor banks.

![Equivalent circuit.](./Images/Equivalent_MSB_LSB.png)


**Dynamic Comparator circuit**
Follows the figure associated with the Dynamic Comparator circuit. The circuit is mainly composed of a differential pair (M0 - M4), a latch circuit (M5 -M10), two inverter buffers, and a NAND to generate the output signal of the comparator. 

![Dynamic Comparator.](./Images/Dynamic_Comparator.png)

When clks = '0', which is the reset phase of the Dynamic Comparator, transistor M0 is off, and transistors M3 and M4 load the nodes ap and an to VDD. Consequently, transistors M11 and M12 ground the nodes dp and dn. Thus the output of the NAND gate is equal to '0'.

When clks = '1', transistor M0 is on, whereas transistors M3 and M4 are off. Since transistors M1 and M2 are on, current passes through them and nodes ap and an discharge to GND. Transistors M11 and M12 are off, thus nodes dp and dn are no longer forced to ground. 

If vdp > vdn, node an approaches GND faster than ap. This voltage difference is detected by transistors M7 and M8 to set one of the outputs of the latch to VDD and the other to GND. Buffers are placed at the output of the latch to have the operate with the same output capacitances.

**Asyncronous SAR logic**

Follow the figure associated with the main control that implements the SAR logic.

When clks = '0', the asynchronous operation starts The valid signal generated by the dynamic comparator enables the flip-flops that integrate the main control logic. The asynchronous operation activates the MSB to LSB bit decision circuits in a cascading effect.

When clks = '1', the main control logic is reset for a new conversion cycle, thus the flip-flop outputs are grounded.

![SAR main control logic schematic.](./Images/Main_Control_Logic.png)

The signals CLK1 - CLK11 activate the 11 cells that control the differential DAC. Follows the figure associated with one of these 11 cells.

The sampling phase starts when CK11 = '0'. The switches TGa and TGb disconnect the circuit associated with DAC control. TG connects the lower plate of the capacitor to Vcm. S1 and S2 turn on, whereas S3 and S4 are off. At this point, the asynchronous logic is disabled, and Valid = clkc = '0'.

When the sampling phase ends (falling edge of clks), the sampling switches are disconnected and starts the asynchronous operation. After a certain delay, the dynamic comparator clock is set to '1', then an initial comparison between the samples obtained by the differential signal will occur without the switching of the lower plates of the capacitors that implement the DAC.

When the outputs of the Dynamic Comparator are set to a specific logic level, Valid is set to '1'. Then DFF-11 in the main control logic is triggered by the rising edge of Valid. CK11 passes from '0' to '1', and then DFF-C is also triggered. The data generated by the Dynamic Comparator vocp (or vocn) goes to D0 and S1, SS2 and TG are off. 

TGa and TGb connect node D0 to the gates of S3 and S4. Depending on the value of vocp (or vocn) the lower plate of the capacitor goes to VDD or GND through S3 and S4, respectively.

After the Valid signal is generated by the Dynamic Comparator, clkc = '0' after a certain delay defined by the logic gates that are part of the asynchronous clock generator, the Dynamic Comparator is reset. The falling edge of clkc forces the outputs of Dynamic Comparator to VDD, and then Valid = '0'. After some delay, clkc comes back to '1', and then a new decision is generated by the Dynamic Comparator based on the voltage values set at the outputs of the DAC.

Once all the 11 bits are obtained, in the next rising edge of clks, the DFF-R generates the output value of the 11 bits obtained by the SAR algorithm. Then, a new conversion cycle starts.

![Asynchronous SAR logic schematic.](./Images/SAR_async_logic.png)

**Generation of the asynchronous clock**

Follows the figures associated with the circuit that generates the asynchronous clock and the respective time diagram.

The falling edge of clks starts the first comparison during the time td1, defined by the delay of the NOR gate, the delay cell, and the inverter. When the comparison ends, Valid is generated after a time td2 associated with the delay of the Dynamic Comparator, the output buffers, and the NAND gate. Once Valid is generated, clkc takes the value of '0' after a time td3 that corresponds to the delays of the NOR gate the charging time of the delay cell, and the inverter. Thus, the clock of the dynamic comparator clkc depends on the values of the signals clks, CK1, and Valid. 

Td4 corresponds to the reset time of the Dynamic Comparator, the delay of the output buffers of the Dynamic Comparator, delay of the NAND gates. This time interval occurs 11 times to obtain the 11 bits, then after the rising edge of CLK1, the conversion ends. The decision time of the Dynamic Comparator is td2 + td3, whereas td1 + td4 is associated with the reset phase.

To compensate for PVT variations in the generation of the asynchronous clock, we use a variable delay cell to obtain the 11 decisions for the 11 bits.

![Delay cell schematic.](./Images/Delay_cell.png)

### Layout techniques implemented

To guarantee a correct operation of the physical implementation (layout) of the ADC and overcome the possible undesired effects mentioned above, we use the following layout techniques:

1) Common centroid
2) Dummy transistors
3) Symmetrical architecture.


In [ ]:
%%writefile .spiceinit
set ngbehavior=hs

### Bootstrap simulations

Follows the testbench implemented to test the bootstrap technique.

![Bootstrap testbench.](./Images/Bootstrap_tb.png)

### Simulation code of the bootstrap technique

In [ ]:
%%writefile netlist.spice
*Models

.include "globalfoundries-pdk-libs-gf180mcu_fd_pr/models/ngspice/design.ngspice"
.lib "globalfoundries-pdk-libs-gf180mcu_fd_pr/models/ngspice/sm141064.ngspice" typical
.lib "globalfoundries-pdk-libs-gf180mcu_fd_pr/models/ngspice/smbb000149.ngspice" typical

XM1 net1 clks VDD VDD pfet_03v3 L=0.28e-6 W=0.22e-6 nf=1 ad='int((nf+1)/2) * W/nf * 0.18u' as='int((nf+2)/2) * W/nf * 0.18u'
+ pd='2*int((nf+1)/2) * (W/nf + 0.18u)' ps='2*int((nf+2)/2) * (W/nf + 0.18u)' nrd='0.18u / W' nrs='0.18u / W'
+ sa=0 sb=0 sd=0 m=1
XM2 net1 clks net2 GND nfet_03v3 L=0.28e-6 W=0.22e-6 nf=1 ad='int((nf+1)/2) * W/nf * 0.18u' as='int((nf+2)/2) * W/nf * 0.18u'
+ pd='2*int((nf+1)/2) * (W/nf + 0.18u)' ps='2*int((nf+2)/2) * (W/nf + 0.18u)' nrd='0.18u / W' nrs='0.18u / W'
+ sa=0 sb=0 sd=0 m=1
XM3 net1 vgg net2 GND nfet_03v3 L=0.28e-6 W=0.22e-6 nf=1 ad='int((nf+1)/2) * W/nf * 0.18u' as='int((nf+2)/2) * W/nf * 0.18u'
+ pd='2*int((nf+1)/2) * (W/nf + 0.18u)' ps='2*int((nf+2)/2) * (W/nf + 0.18u)' nrd='0.18u / W' nrs='0.18u / W'
+ sa=0 sb=0 sd=0 m=1
XM4 VDD vgg vb vb pfet_03v3 L=0.28e-6 W=0.22e-6 nf=1 ad='int((nf+1)/2) * W/nf * 0.18u' as='int((nf+2)/2) * W/nf * 0.18u'
+ pd='2*int((nf+1)/2) * (W/nf + 0.18u)' ps='2*int((nf+2)/2) * (W/nf + 0.18u)' nrd='0.18u / W' nrs='0.18u / W'
+ sa=0 sb=0 sd=0 m=1
XM5 vgg net1 vb vb pfet_03v3 L=0.28e-6 W=0.22e-6 nf=1 ad='int((nf+1)/2) * W/nf * 0.18u' as='int((nf+2)/2) * W/nf * 0.18u'
+ pd='2*int((nf+1)/2) * (W/nf + 0.18u)' ps='2*int((nf+2)/2) * (W/nf + 0.18u)' nrd='0.18u / W' nrs='0.18u / W'
+ sa=0 sb=0 sd=0 m=1
XM6 vgg VDD net3 GND nfet_03v3 L=0.28e-6 W=0.22e-6 nf=1 ad='int((nf+1)/2) * W/nf * 0.18u' as='int((nf+2)/2) * W/nf * 0.18u'
+ pd='2*int((nf+1)/2) * (W/nf + 0.18u)' ps='2*int((nf+2)/2) * (W/nf + 0.18u)' nrd='0.18u / W' nrs='0.18u / W'
+ sa=0 sb=0 sd=0 m=1
XM7 net3 n_clks GND GND nfet_03v3 L=0.28e-6 W=0.22e-6 nf=1 ad='int((nf+1)/2) * W/nf * 0.18u' as='int((nf+2)/2) * W/nf * 0.18u'
+ pd='2*int((nf+1)/2) * (W/nf + 0.18u)' ps='2*int((nf+2)/2) * (W/nf + 0.18u)' nrd='0.18u / W' nrs='0.18u / W'
+ sa=0 sb=0 sd=0 m=1
XM8 vinp vgg net2 GND nfet_03v3 L=0.28e-6 W=0.22e-6 nf=1 ad='int((nf+1)/2) * W/nf * 0.18u' as='int((nf+2)/2) * W/nf * 0.18u'
+ pd='2*int((nf+1)/2) * (W/nf + 0.18u)' ps='2*int((nf+2)/2) * (W/nf + 0.18u)' nrd='0.18u / W' nrs='0.18u / W'
+ sa=0 sb=0 sd=0 m=1
XM9 net2 n_clks GND GND nfet_03v3 L=0.28e-6 W=0.22e-6 nf=1 ad='int((nf+1)/2) * W/nf * 0.18u' as='int((nf+2)/2) * W/nf * 0.18u'
+ pd='2*int((nf+1)/2) * (W/nf + 0.18u)' ps='2*int((nf+2)/2) * (W/nf + 0.18u)' nrd='0.18u / W' nrs='0.18u / W'
+ sa=0 sb=0 sd=0 m=1
XM10 vdp vgg vinp GND nfet_03v3 L=0.28e-6 W=39e-6 nf=1 ad='int((nf+1)/2) * W/nf * 0.18u' as='int((nf+2)/2) * W/nf * 0.18u'
+ pd='2*int((nf+1)/2) * (W/nf + 0.18u)' ps='2*int((nf+2)/2) * (W/nf + 0.18u)' nrd='0.18u / W' nrs='0.18u / W'
+ sa=0 sb=0 sd=0 m=1
C1 vb net2 10e-12 m=1
C2 vdp GND 10e-12 m=1
V5 clks GND PULSE(0 3.3 100e-9 200e-9 200e-9 500e-9 1e-6 2)
.save i(v5)
V6 n_clks GND PULSE(3.3 0 100e-9 50e-9 50e-9 500e-9 1e-6 2)
.save i(v6)
V1 VDD GND 3.3
.save i(v1)
V2 net4 GND 0
.save i(v2)
V3 vinp net4 SIN(0 1 1e6) DC 0
.save i(v3)
**** begin user architecture code



*Parameters
.param vds = 3.3
.param vgs = 1.65

.options TEMP = 50.0


*Data to save
.save all  @M.xm1.m0[vgs]  @M.xm2.m0[vgs]  @M.xm3.m0[vgs]  @M.xm4.m0[vgs]  @M.xm5.m0[vgs]  @M.xm6.m0[vgs]  @M.xm7.m0[vgs]  @M.xm8.m0[vgs]  @M.xm9.m0[vgs]

*Simulation
.control

.ic vb = 0
.ic vdp = 0
tran 0.01e-9 2e-6 uic
setplot tran1
plot v(clks) v(vb) v(vdp)
*print @M.xm9.m0[vgs]
write bootstrap_tran.raw

reset
op
save all
set filetype = binary
*write bootstrap_dc.raw
wrdata output_bootstrap_dc.txt @M.xm1.m0[vgs] @M.xm2.m0[vgs] @M.xm3.m0[vgs] @M.xm4.m0[vgs]
+ @M.xm5.m0[vgs] @M.xm6.m0[vgs] @M.xm7.m0[vgs] @M.xm8.m0[vgs] @M.xm9.m0[vgs]

.endc
.end


**** end user architecture code
**.ends
.GLOBAL GND
.end

In [ ]:
!ngspice -b netlist.spice

In [ ]:
import math
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
df = pd.read_csv("output_bootstrap_dc.txt", delim_whitespace=True, header=None)
df.columns = ["dc_1","M.xm1.m0[vgs]","dc_2","M.xm2.m0[vgs]", "dc_3","M.xm3.m0[vgs]", "dc_4","M.xm4.m0[vgs]", "dc_5","M.xm5.m0[vgs]", "dc_6","M.xm6.m0[vgs]", "dc_7", "M.xm7.m0[vgs]", "dc_8", "M.xm8.m0[vgs]", "dc_9", "M.xm9.m0[vgs]"]
df

Follows the link in google Colab of the simulation:
https://colab.research.google.com/drive/1xFjQYae21Ug-0oE_60KGIUCmoF56BrW0?usp=sharing

### Dynamic Comparator simulations

Follows the testbench implemented to test the Dynamic Comparator.

![Dynamic Comparator testbench.](./Images/Dynamic_Comparator_tb.png)

### Simulation code of the Dynamic Comparator

In [3]:
%%writefile netlist.spice
*Models
.include "globalfoundries-pdk-libs-gf180mcu_fd_pr/models/ngspice/design.ngspice"
.lib "globalfoundries-pdk-libs-gf180mcu_fd_pr/models/ngspice/sm141064.ngspice" typical

XM1 out_b clkc VDD VDD pfet_03v3 L=0.28e-6 W=3.9e-6 nf=1 ad='int((nf+1)/2) * W/nf * 0.18e-6' as='int((nf+2)/2) * W/nf * 0.18u'
+ pd='2*int((nf+1)/2) * (W/nf + 0.18e-6)' ps='2*int((nf+2)/2) * (W/nf + 0.18e-6)' nrd='0.18u / W' nrs='0.18e-6 / W'
+ sa=0 sb=0 sd=0 m=1
XM2 out_a clkc VDD VDD pfet_03v3 L=0.28e-6 W=3.9e-6 nf=1 ad='int((nf+1)/2) * W/nf * 0.18u' as='int((nf+2)/2) * W/nf * 0.18u'
+ pd='2*int((nf+1)/2) * (W/nf + 0.18u)' ps='2*int((nf+2)/2) * (W/nf + 0.18u)' nrd='0.18u / W' nrs='0.18u / W'
+ sa=0 sb=0 sd=0 m=1
XM3 out_a vdp net1 GND nfet_03v3 L=0.28e-6 W=0.9e-6 nf=1 ad='int((nf+1)/2) * W/nf * 0.18u' as='int((nf+2)/2) * W/nf * 0.18u'
+ pd='2*int((nf+1)/2) * (W/nf + 0.18u)' ps='2*int((nf+2)/2) * (W/nf + 0.18u)' nrd='0.18u / W' nrs='0.18u / W'
+ sa=0 sb=0 sd=0 m=1
XM4 out_b vdn net1 GND nfet_03v3 L=0.28e-6 W=0.9e-6 nf=1 ad='int((nf+1)/2) * W/nf * 0.18u' as='int((nf+2)/2) * W/nf * 0.18u'
+ pd='2*int((nf+1)/2) * (W/nf + 0.18u)' ps='2*int((nf+2)/2) * (W/nf + 0.18u)' nrd='0.18u / W' nrs='0.18u / W'
+ sa=0 sb=0 sd=0 m=1
XM5 net1 clkc GND GND nfet_03v3 L=0.28e-6 W=10e-6 nf=1 ad='int((nf+1)/2) * W/nf * 0.18u' as='int((nf+2)/2) * W/nf * 0.18u'
+ pd='2*int((nf+1)/2) * (W/nf + 0.18u)' ps='2*int((nf+2)/2) * (W/nf + 0.18u)' nrd='0.18u / W' nrs='0.18u / W'
+ sa=0 sb=0 sd=0 m=1
V1 VDD GND 3.3
.save i(v1)
XM6 net2 dp VDD VDD pfet_03v3 L=0.28e-6 W=16e-6 nf=1 ad='int((nf+1)/2) * W/nf * 0.18u' as='int((nf+2)/2) * W/nf * 0.18u'
+ pd='2*int((nf+1)/2) * (W/nf + 0.18u)' ps='2*int((nf+2)/2) * (W/nf + 0.18u)' nrd='0.18u / W' nrs='0.18u / W'
+ sa=0 sb=0 sd=0 m=1
XM7 net3 dn VDD VDD pfet_03v3 L=0.28e-6 W=16e-6 nf=1 ad='int((nf+1)/2) * W/nf * 0.18u' as='int((nf+2)/2) * W/nf * 0.18u'
+ pd='2*int((nf+1)/2) * (W/nf + 0.18u)' ps='2*int((nf+2)/2) * (W/nf + 0.18u)' nrd='0.18u / W' nrs='0.18u / W'
+ sa=0 sb=0 sd=0 m=1
XM8 dn out_b net2 VDD pfet_03v3 L=0.28e-6 W=16e-6 nf=1 ad='int((nf+1)/2) * W/nf * 0.18u' as='int((nf+2)/2) * W/nf * 0.18u'
+ pd='2*int((nf+1)/2) * (W/nf + 0.18u)' ps='2*int((nf+2)/2) * (W/nf + 0.18u)' nrd='0.18u / W' nrs='0.18u / W'
+ sa=0 sb=0 sd=0 m=1
XM9 dp out_a net3 VDD pfet_03v3 L=0.28e-6 W=16e-6 nf=1 ad='int((nf+1)/2) * W/nf * 0.18u' as='int((nf+2)/2) * W/nf * 0.18u'
+ pd='2*int((nf+1)/2) * (W/nf + 0.18u)' ps='2*int((nf+2)/2) * (W/nf + 0.18u)' nrd='0.18u / W' nrs='0.18u / W'
+ sa=0 sb=0 sd=0 m=1
XM14 vocp dp VDD VDD pfet_03v3 L=0.28e-6 W=2.8e-6 nf=1 ad='int((nf+1)/2) * W/nf * 0.18u' as='int((nf+2)/2) * W/nf * 0.18u'
+ pd='2*int((nf+1)/2) * (W/nf + 0.18u)' ps='2*int((nf+2)/2) * (W/nf + 0.18u)' nrd='0.18u / W' nrs='0.18u / W'
+ sa=0 sb=0 sd=0 m=1
XM15 vocp dp GND GND nfet_03v3 L=0.28e-6 W=1e-6 nf=1 ad='int((nf+1)/2) * W/nf * 0.18u' as='int((nf+2)/2) * W/nf * 0.18u'
+ pd='2*int((nf+1)/2) * (W/nf + 0.18u)' ps='2*int((nf+2)/2) * (W/nf + 0.18u)' nrd='0.18u / W' nrs='0.18u / W'
+ sa=0 sb=0 sd=0 m=1
XM16 vocn dn VDD VDD pfet_03v3 L=0.28e-6 W=2.8e-6 nf=1 ad='int((nf+1)/2) * W/nf * 0.18u' as='int((nf+2)/2) * W/nf * 0.18u'
+ pd='2*int((nf+1)/2) * (W/nf + 0.18u)' ps='2*int((nf+2)/2) * (W/nf + 0.18u)' nrd='0.18u / W' nrs='0.18u / W'
+ sa=0 sb=0 sd=0 m=1
XM17 vocn dn GND GND nfet_03v3 L=0.28e-6 W=1e-6 nf=1 ad='int((nf+1)/2) * W/nf * 0.18u' as='int((nf+2)/2) * W/nf * 0.18u'
+ pd='2*int((nf+1)/2) * (W/nf + 0.18u)' ps='2*int((nf+2)/2) * (W/nf + 0.18u)' nrd='0.18u / W' nrs='0.18u / W'
+ sa=0 sb=0 sd=0 m=1
XM18 net4 vocn GND GND nfet_03v3 L=0.28e-6 W=1e-6 nf=1 ad='int((nf+1)/2) * W/nf * 0.18u' as='int((nf+2)/2) * W/nf * 0.18u'
+ pd='2*int((nf+1)/2) * (W/nf + 0.18u)' ps='2*int((nf+2)/2) * (W/nf + 0.18u)' nrd='0.18u / W' nrs='0.18u / W'
+ sa=0 sb=0 sd=0 m=1
XM19 Valid vocp net4 GND nfet_03v3 L=0.28e-6 W=1e-6 nf=1 ad='int((nf+1)/2) * W/nf * 0.18u' as='int((nf+2)/2) * W/nf * 0.18u'
+ pd='2*int((nf+1)/2) * (W/nf + 0.18u)' ps='2*int((nf+2)/2) * (W/nf + 0.18u)' nrd='0.18u / W' nrs='0.18u / W'
+ sa=0 sb=0 sd=0 m=1
XM20 Valid vocp VDD VDD pfet_03v3 L=0.28e-6 W=2.8e-6 nf=1 ad='int((nf+1)/2) * W/nf * 0.18u' as='int((nf+2)/2) * W/nf * 0.18u'
+ pd='2*int((nf+1)/2) * (W/nf + 0.18u)' ps='2*int((nf+2)/2) * (W/nf + 0.18u)' nrd='0.18u / W' nrs='0.18u / W'
+ sa=0 sb=0 sd=0 m=1
XM21 Valid vocn VDD VDD pfet_03v3 L=0.28e-6 W=2.8e-6 nf=1 ad='int((nf+1)/2) * W/nf * 0.18u' as='int((nf+2)/2) * W/nf * 0.18u'
+ pd='2*int((nf+1)/2) * (W/nf + 0.18u)' ps='2*int((nf+2)/2) * (W/nf + 0.18u)' nrd='0.18u / W' nrs='0.18u / W'
+ sa=0 sb=0 sd=0 m=1
V2 net5 GND 2.9
.save i(v2)
V3 net5 vdn 0.2
.save i(v3)
V4 vdp net5 0.2
.save i(v4)
V5 clkc GND PULSE(0 3.1 100e-9 50e-9 50e-9 500e-9 1000e-9 2)
.save i(v5)
V6 n_clkc GND PULSE(3.1 0 100e-9 50e-9 50e-9 500e-9 1000e-9 2)
.save i(v6)
XM10 dp dn GND GND nfet_03v3 L=0.28e-6 W=6.2e-6 nf=1 ad='int((nf+1)/2) * W/nf * 0.18u' as='int((nf+2)/2) * W/nf * 0.18u'
+ pd='2*int((nf+1)/2) * (W/nf + 0.18u)' ps='2*int((nf+2)/2) * (W/nf + 0.18u)' nrd='0.18u / W' nrs='0.18u / W'
+ sa=0 sb=0 sd=0 m=1
XM11 dn n_clkc GND GND nfet_03v3 L=0.28e-6 W=3.1e-6 nf=1 ad='int((nf+1)/2) * W/nf * 0.18u' as='int((nf+2)/2) * W/nf * 0.18u'
+ pd='2*int((nf+1)/2) * (W/nf + 0.18u)' ps='2*int((nf+2)/2) * (W/nf + 0.18u)' nrd='0.18u / W' nrs='0.18u / W'
+ sa=0 sb=0 sd=0 m=1
XM12 dn dp GND GND nfet_03v3 L=0.28e-6 W=6.2e-6 nf=1 ad='int((nf+1)/2) * W/nf * 0.18u' as='int((nf+2)/2) * W/nf * 0.18u'
+ pd='2*int((nf+1)/2) * (W/nf + 0.18u)' ps='2*int((nf+2)/2) * (W/nf + 0.18u)' nrd='0.18u / W' nrs='0.18u / W'
+ sa=0 sb=0 sd=0 m=1
XM13 dp n_clkc GND GND nfet_03v3 L=0.28e-6 W=3.1e-6 nf=1 ad='int((nf+1)/2) * W/nf * 0.18u' as='int((nf+2)/2) * W/nf * 0.18u'
+ pd='2*int((nf+1)/2) * (W/nf + 0.18u)' ps='2*int((nf+2)/2) * (W/nf + 0.18u)' nrd='0.18u / W' nrs='0.18u / W'
+ sa=0 sb=0 sd=0 m=1
**** begin user architecture code



*Parameters
.param vds = 3.3
.param vgs = 1.65

.options TEMP = 50.0

*Data to save
.save all

*Simulation
.control

tran 0.01e-9 2e-6
setplot tran1
plot v(clkc) v(vdp) v(vdn)
plot v(out_a) v(out_b)
plot v(dp) v(dn)
plot v(vocp) v(vocn)
plot v(Valid)
*write dynamic_comp_tran.raw
wrdata output_dynamic_comparator.txt v(clkc) v(vdp) v(vdn) v(out_a) v(out_b) v(vocp) v(vocn) v(Valid)

reset
op
save all
set filetype = binary
write dynamic_comp_dc.raw

.endc
.end


**** end user architecture code
**.ends
.GLOBAL GND
.end

Overwriting netlist.spice


In [ ]:
!ngspice -b netlist.spice

In [ ]:
import math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.image as mpimg

#df = pd.read_csv("output_dynamic_comparator.txt", delim_whitespace=True, header=None)
#df.columns = ["time","clkc","time_1","vdp", "time_2","vdn", "time_3","out_a", "time_4","out_b", "time_5","vocp", "time_6", "vocn", "time_7", "Valid"]
#df

print(pd.read_csv)
df = pd.read_csv("output_dynamic_comparator.txt", delim_whitespace=True, header=None)
a=df.to_numpy()
#a=df.to_numpy().tolist()

In [ ]:
time=[]
clkc=[]
vdp=[]
vdn=[]
out_a=[]	
out_b=[]
vocp=[]	
vocn=[]
valid=[]

i=0;
k=0;
while k <= len(a)-1:
  while i <= 15:
    if(i==0):
      time.append(a[k][i]);
    elif(i==1):
      clkc.append(a[k][i]);
    elif(i==3):
      vdp.append(a[k][i]);
    elif(i==5):
      vdn.append(a[k][i]);
    elif(i==7):
      out_a.append(a[k][i]);
    elif(i==9):
      out_b.append(a[k][i]);
    elif(i==11):
      vocp.append(a[k][i]);
    elif(i==13):
      vocn.append(a[k][i]);
    elif(i==15):
      valid.append(a[k][i]);
  
    i=i+1
  i=0
  k=k+1;

In [ ]:
plt.plot(time,clkc)
plt.plot(time,vdp)
plt.plot(time,vdn)

In [ ]:
plt.plot(time,out_a)
plt.plot(time,out_b)

In [ ]:
plt.plot(time,vocp)
plt.plot(time,vocn)

In [1]:
plt.plot(time,valid)

NameError: name 'plt' is not defined

Since vdp > vdn, out_a is going to be faster than out_b, and then it reaches first ground. Then the latch forces vocp to go to Vdd, a vocn to ground (when the clock is '1'). The latter demonstrates the correct behavior of the Dynamic Comparator. 

Follows the link of the simulations in google colab:
https://colab.research.google.com/drive/1ZHKam9tu841AG60T2CkDXxoguhZH8Bqs?usp=sharing

### Simulation code of Asynchronous SAR logic

Since we do not have experience running the mixed-signal (MS) flow with xschem, we implement the flow used in tiny tapeout (http://tinytapeout.com/) (Openlane) with gf180m. To validate the performance of the code, we use cocotb (https://www.cocotb.org/) to test our code with Python. We have a little bit of experience with eSim (https://esim.fossee.in/mixed-signal-design-marathon) to run mixed signals, but we do not know if eSim is going to be the platform to use during the PICO contest, we decided to present the simulations of the asynchronous SAR logic with the flow used in tiny tapeout. One of the main tasks of our proposal is to determine the tools used in the PICO contest to run the MS flow and get familiar with it.

## SAR ADC code

In [1]:
`default_nettype none
`timescale 1ns/1ps

/////// SAR_control ///////

module SAR_control(

	io_in,
	io_out

);

	input wire [2:0] io_in;
	output wire [21:0] io_out;
	
	wire q_dff_11_2_d_dff_10;
	wire q_dff_10_2_d_dff_9;
	wire q_dff_9_2_d_dff_8;
	wire q_dff_8_2_d_dff_7;	
	wire q_dff_7_2_d_dff_6;
	wire q_dff_6_2_d_dff_5;	
	wire q_dff_5_2_d_dff_4;
	wire q_dff_4_2_d_dff_3;	
	wire q_dff_3_2_d_dff_2;
	wire q_dff_2_2_d_dff_1;
		
				
	assign io_out[0] = q_dff_11_2_d_dff_10;
	assign io_out[2] = q_dff_10_2_d_dff_9;
	assign io_out[4] = q_dff_9_2_d_dff_8;
	assign io_out[6] = q_dff_8_2_d_dff_7;
	assign io_out[8] = q_dff_7_2_d_dff_6;
	assign io_out[10] = q_dff_6_2_d_dff_5;
	assign io_out[12] = q_dff_5_2_d_dff_4;
	assign io_out[14] = q_dff_4_2_d_dff_3;
	assign io_out[16] = q_dff_3_2_d_dff_2;
	assign io_out[18] = q_dff_2_2_d_dff_1;	
	
	dff_reset dff_reset_11 (
		.d(io_in[0]),
		.clk(io_in[1]),
		.rst(io_in[2]),
		.q(q_dff_11_2_d_dff_10),
		.nq(io_out[1])		
	);
	
	dff_reset dff_reset_10 (
		.d(q_dff_11_2_d_dff_10),
		.clk(io_in[1]),
		.rst(io_in[2]),
		.q(q_dff_10_2_d_dff_9),
		.nq(io_out[3])		
	);
	
	dff_reset dff_reset_9 (
		.d(q_dff_10_2_d_dff_9),
		.clk(io_in[1]),
		.rst(io_in[2]),
		.q(q_dff_9_2_d_dff_8),
		.nq(io_out[5])		
	);

	dff_reset dff_reset_8 (
		.d(q_dff_9_2_d_dff_8),
		.clk(io_in[1]),
		.rst(io_in[2]),
		.q(q_dff_8_2_d_dff_7),
		.nq(io_out[7])
	);
	
	dff_reset dff_reset_7 (
		.d(q_dff_8_2_d_dff_7),
		.clk(io_in[1]),
		.rst(io_in[2]),
		.q(q_dff_7_2_d_dff_6),
		.nq(io_out[9])			
	);
	
	dff_reset dff_reset_6 (
		.d(q_dff_7_2_d_dff_6),
		.clk(io_in[1]),
		.rst(io_in[2]),
		.q(q_dff_6_2_d_dff_5),
		.nq(io_out[11])			
	);

	dff_reset dff_reset_5 (
		.d(q_dff_6_2_d_dff_5),
		.clk(io_in[1]),
		.rst(io_in[2]),
		.q(q_dff_5_2_d_dff_4),
		.nq(io_out[13])			
	);

	dff_reset dff_reset_4 (
		.d(q_dff_5_2_d_dff_4),
		.clk(io_in[1]),
		.rst(io_in[2]),
		.q(q_dff_4_2_d_dff_3),
		.nq(io_out[15])			
	);

	dff_reset dff_reset_3 (
		.d(q_dff_4_2_d_dff_3),
		.clk(io_in[1]),
		.rst(io_in[2]),
		.q(q_dff_3_2_d_dff_2),
		.nq(io_out[17])			
	);
	
	dff_reset dff_reset_2 (
		.d(q_dff_3_2_d_dff_2),
		.clk(io_in[1]),
		.rst(io_in[2]),
		.q(q_dff_2_2_d_dff_1),
		.nq(io_out[19])			
	);

	dff_reset dff_reset_1 (
		.d(q_dff_2_2_d_dff_1),
		.clk(io_in[1]),
		.rst(io_in[2]),
		.q(io_out[20]),
		.nq(io_out[21])			
	);			
endmodule

module dff_reset(
	d,
	clk,
	rst,
	q,
	nq
);

input wire d;
input wire clk;
input wire rst;
output wire q;
output wire nq;

reg temp;
always @(posedge clk, posedge rst) begin
	if(rst == 1'b1)
	begin
	temp <= 1'b0;
	end
	else begin
  	temp <= d;
  	end
end

assign q = temp;
assign nq = ~temp;

endmodule

SyntaxError: invalid decimal literal (263895664.py, line 2)

## Test-bench code

In [2]:
`default_nettype none
`timescale 1ns/1ps

module tb(
		input vddd,
		input valid,
		input clks,
		output q_11,
		output nq_11,
		output q_10,
		output nq_10,
		output q_9,
		output nq_9,
		output q_8,
		output nq_8,
		output q_7,
		output nq_7,
		output q_6,
		output nq_6,
		output q_5,
		output nq_5,
		output q_4,
		output nq_4,
		output q_3,
		output nq_3,
		output q_2,
		output nq_2,
		output q_1,
		output nq_1		
	);
	
	initial begin
		$dumpfile ("tb.vcd");
		$dumpvars (0,tb);
		#1;
	end
	
	wire [7:0] inputs = {5'b0, clks, valid, vddd};
	wire [21:0] outputs;
	assign q_11 = outputs[0];
	assign nq_11 = outputs[1];
	assign q_10 = outputs[2];
	assign nq_10 = outputs[3];
	assign q_9 = outputs[4];
	assign nq_9 = outputs[5];
	assign q_8 = outputs[6];
	assign nq_8 = outputs[7];	
	assign q_7 = outputs[8];
	assign nq_7 = outputs[9];
	assign q_6 = outputs[10];
	assign nq_6 = outputs[11];
	assign q_5 = outputs[12];
	assign nq_5 = outputs[13];
	assign q_4 = outputs[14];
	assign nq_4 = outputs[15];
	assign q_3 = outputs[16];
	assign nq_3 = outputs[17];
	assign q_2 = outputs[18];
	assign nq_2 = outputs[19];	
	assign q_1 = outputs[20];
	assign nq_1 = outputs[21];

	SAR_control SAR_control_0(
	.io_in (inputs),
	.io_out (outputs)
	);
	
endmodule

SyntaxError: invalid decimal literal (2807825532.py, line 2)

## Makefile

In [ ]:
SIM ?= icarus
TOPLEVEL_LANG ?= verilog

VERILOG_SOURCES += $(PWD)/tb.v $(PWD)/SAR_control.v

TOPLEVEL = tb

MODULE = test

include $(shell cocotb-config --makefiles)/Makefile.sim

## Test code

In [ ]:
import cocotb
from cocotb.triggers import Timer

@cocotb.test()

async def SAR_control(dut):
    dut._log.info("START")
   
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(10, 'ns')
# INIT
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(10, 'ns')

    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 1
    await cocotb.triggers.Timer(10, 'ns')

    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(10, 'ns')

#Pulse 1    
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 1
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')

#Pulse 2    
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 1
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    

#Pulse 3    
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 1
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
#Pulse 4
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 1
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')

#Pulse 5    
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 1
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')

#Pulse 6    
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 1
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')

#Pulse 7    
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 1
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')

#Pulse 8    
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 1
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')

#Pulse 9    
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 1
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')

#Pulse 10   
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 1
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')

#Pulse 11
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 1
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')


# INIT
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(10, 'ns')

    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 1
    await cocotb.triggers.Timer(10, 'ns')

    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(10, 'ns')

#Pulse 1    
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 1
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')

#Pulse 2    
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 1
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    

#Pulse 3    
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 1
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
#Pulse 4
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 1
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')

#Pulse 5    
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 1
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')

#Pulse 6    
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 1
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')

#Pulse 7    
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 1
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')

#Pulse 8    
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 1
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')

#Pulse 9    
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 1
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')

#Pulse 10   
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 1
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')

#Pulse 11
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 1
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')
    
    dut.vddd.value = 1
    dut.valid.value = 0
    dut.clks.value = 0
    await cocotb.triggers.Timer(1, 'ns')


Follows the simulation obtained in the software gtkwave.

![SAR Logic Simulation](./Images/Sim_ADC_SAR_logica.png)

Follow the layout obtained in Magic after the flow ends.
![SAR Logic Layout](./Images/SAR_Logic_layout.jpg)

In the next figure, we present the zoom of one corner of the layout, where the different standard cells are placed.
![SAR Logic Layout_zoom](./Images/SAR_Logic_layout_zoom.jpg)



## III. Summary of your Idea

As mentioned before, we propose an 11-bit differential SAR-ADC with a bootstrapping technique for sampling and an asynchronous logic for the PICO contest. An adequate characterization of the ENOB of the track and hold circuit since it determines the ENOB of the ADC. A strong study must be done to understand the mixed signal flow with Xschem. Finally, layout techniques are mandatory, such as common centroid, interdigitated, and symmetry to mitigate undesired effects, and maintain both inputs identical as possible.

## IV. Planification and Tasks

Follows the planification and tasks of our proposal:

|Month| Tasks to be developed|||
|:--:|:--:|:----------:|:----------:|
|May||||
|Week 2: 8 - 12|Proposal deadline (May 8)|||
|Week 3: 15 - 19|Annoucement of selected teams||
|Week 4: 22 - 26|||
|June|Start of online meetups|||
|Week 1: May 29 - June 2| 1) Specifications Verification. 2) Possible changes?||
|Week 2: 5 - 9| 1) Optimization of the bootstrap circuit & the dynamic comparator. (schematic) 2) Redesign of the proposal if changes.||
|Week 3: 12 - 16| 1) Optimization of the bootstrap circuit & the dynamic comparator. (schematic) 2) Redesign of the proposal if changes.||
|Week 4: 19 - 23| 1) Optimization of the bootstrap circuit & the dynamic comparator.  (schematic) 2) Redesign of the proposal if changes.||
|Week 5: 26 - 30| 1) Implementation of the DAC circuit.  (schematic) 2) Bootstrap circuit. (layout)||
|July||||
|Week 1: 3 - 7| 1) Implementation of the DAC circuit logic.  (schematic) 2) Bootstrap circuit. (layout)||
|Week 2: 10 - 14| 1) Implementation of the asynchronous SAR logic. (schematic) 2) Bootstrap circuit. (layout) 3) Integration Boostrap + DAC (schematic)||
|Week 3: 12 - 16| 1) Implementation of the asynchronous SAR logic. (schematic)  2) Dynamic comparator. (layout) 3) Integration Boostrap + DAC (schematic)||
|Week 4: 17 - 21| 1) Implementation of the asynchronous SAR logic. (schematic) 2) Dynamic comparator. (layout) 3) Integration Boostrap + DAC (B-DAC) (schematic)||
|Week 5: 24 - 28| 1) Mixed-Signal flow study.  2) DAC circuit. (layout) 3) Integration B-DAC + Dynamic comparator (B-DAC-DC) (schematic)||
|August||||
|Week 1: July 31 - August 4| 1) Mixed-Signal flow study.  2) DAC circuit. (layout) 3) Integration B-DAC + Dynamic comparator (B-DAC-DC) (schematic)||
|Week 2: 7 - 11|1) Mixed-Signal flow study.  2) DAC circuit. (layout) 3) Integration B-DAC + Dynamic comparator (B-DAC-DC)  (schematic)||
|Week 3: 14 - 18| 1) ADC full integration (schematic) 2) ||
|Week 4: 21 - 25| 1) ADC full integration (schematic)  ||
|Week 5: August 28 - September 1| 1) ADC full integration. (schematic) 2) Integration Boostrap + DAC. (layout)||
|September||||
|Week 1: 4 - 8| 1) ADC full integration. (schematic) 2) Integration Boostrap + DAC. (layout)||
|Week 2: 11 - 15| 1) ADC full integration. (schematic) 2) Integration Boostrap + DAC. (layout)||
|Week 3: 18 - 22| 1) ADC full integration. (schematic) 2) Integration B-DAC + DC (layout)||
|Week 4: 25 - 29|1) ADC full integration. (schematic) 2) Integration B-DAC + DC (layout)||
|October||||
|Week 1: 2 - 6| 1) ADC full integration. (layout) ||
|Week 2: 9 - 13| 1) ADC full integration. (layout) ||
|Week 3: 16 - 20| 1) ADC full integration. (layout) ||
|Week 4: 23 - 27| 1) ADC full integration. (layout) ||
|Week 5: October 30 - November3| 1) Redesign? (schematic & layout)||
|November||||
|Week 1: 6 - 10| 1) Redesign? (schematic & layout)||
|Week 2: 13 - 17| 1) Redesign? (schematic & layout)||
|Week 3: 20 - 27|1) Delivery gds of the final layout||
|Week 4: November 27 - December 1|||
|December | Tapeout deadline|||



